In [5]:
import os
print(" Current working directory:", os.getcwd())

 Current working directory: C:\Users\DELL\Desktop\Horizon17\sopcompliance


In [2]:
pip install pyPDF2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import PyPDF2
import nltk
from nltk.tokenize import sent_tokenize

In [7]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from PDF while removing headers and footers."""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            
            # Remove known headers and footers
            page_text = re.sub(r"ISO/IEC\s27002:2022\(E\)", "", page_text)
            page_text = re.sub(r"ISO/IEC\s2022\s-\sAll rights reserved", "", page_text)
            page_text = re.sub(r"@ ISO/IEC\s2022\s-\sAll\srights\sreserved", "", page_text)
            page_text = re.sub(r"Page\s\d+", "", page_text)  # Removes page numbers
            
            text += page_text + " "
        return text

In [18]:
def parse_sections(text):
    """Parses sections using regex pattern matching."""
    sections = ["Control", "Purpose", "Guidance"]
    structured_data = {}

    # Define regex pattern for clause identification
    clause_pattern = re.compile(r"(\d+\.\d+)\s(.+?)\s*(?:Control)", re.MULTILINE)

    matches = clause_pattern.finditer(text)

    for match in matches:
        clause_number = match.group(1)
        clause_name = match.group(2).strip()

        temp_content = {"Clause Name": clause_name, "Control": "", "Purpose": "", "Guidance": ""}

        # Extract Control, Purpose, and Guidance using regex with `re.DOTALL` to capture multi-line content
        for section in sections:
            section_pattern = re.compile(rf"{section}\s*(.*?)\s*(?:Purpose|Guidance|Control|[5-8]\.\d+|$)", re.MULTILINE | re.DOTALL)
            section_matches = section_pattern.findall(text[match.end():])

            if section_matches:
                temp_content[section] = " ".join([s.strip().replace("\n", " ") for s in section_matches])

        structured_data[clause_number] = temp_content
    
    return structured_data

In [24]:
pdf_path = "ISO 27002.pdf"

In [25]:
raw_text = extract_text_from_pdf(pdf_path)
data = parse_sections(raw_text)

In [26]:
with open("iso27002_extracted.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("Extraction completed. Data saved to iso27002_extracted.json")

Extraction completed. Data saved to iso27002_extracted.json


In [27]:
import json

# Load JSON file
with open("iso27002_extracted.json", "r") as file:
    data = json.load(file)

# Pretty-print JSON
print(json.dumps(data, indent=4))

{
    "0.3": {
        "Clause Name": "",
        "Control": "determination also depends on the manner in which controls s may not always exert the intended or assumed modifying effect. s are categorized as: type is an attribute to view controls from the perspective of when and how the control title:  Short name of the control; :  What the control is; type Information Information security policy and topic-specific policies should be defined, approved by management, type Information Information security roles and responsibilities should be defined and allocated according to the type Information Conflicting duties and conflicting areas of responsibility should be segregated. type Information Management should require all personnel to apply information security in accordance with the type Information The organization should establish and maintain contact with relevant authorities. type Information The organization should establish and maintain contact with special interest groups or other